In [1]:
from src.data.data_utils import query_mlb_db

In [125]:
kukuchi = 579328
jones = 683003

pitcher = kukuchi

In [126]:
query_str = f'''
    SELECT pitch_type, pitch_number, strikes, balls, outs_when_up,
        CASE
            when stand='R' then 1
            else 0
        END AS stand,
        CASE
            when on_1b is not null then 1
            else 0
        END AS on_1b,
        CASE
            when on_2b is not null then 1
            else 0
        END AS on_2b,
        CASE
            when on_3b is not null then 1
            else 0
        END AS on_3b,
        CASE
            when fld_score - bat_score > 0 then 1
            when fld_score - bat_score = 0 then 0
            else -1
        END AS is_winning,
        LAG(pitch_type) OVER (PARTITION BY game_pk, pitcher, at_bat_number ORDER BY pitch_number) AS prev_pitch,
        ROW_NUMBER() OVER (PARTITION BY game_pk, pitcher ORDER BY at_bat_number, pitch_number) AS cumulative_pitch_number
    FROM Statcast
    WHERE pitcher = {pitcher}
        AND pitch_type IS NOT NULL
        and pitch_type <> 'PO'
        AND game_type <> 'E' || 'S'
    ORDER BY game_pk, at_bat_number, pitch_number
'''
df = query_mlb_db(query_str)

In [127]:
df

,pitch_type,pitch_number,strikes,balls,outs_when_up,stand,on_1b,on_2b,on_3b,is_winning,prev_pitch,cumulative_pitch_number
0,FF,1,0,0,0,1,0,0,0,1,None,1
1,FF,2,1,0,0,1,0,0,0,1,FF,2
2,CU,3,2,0,0,1,0,0,0,1,FF,3
3,FF,1,0,0,1,1,0,0,0,1,None,4
4,FF,2,0,1,1,1,0,0,0,1,FF,5
...,...,...,...,...,...,...,...,...,...,...,...,...
11751,FF,2,1,0,0,1,0,0,0,-1,CU,17
11752,FF,1,0,0,1,0,0,0,0,-1,None,18
11753,SL,2,0,1,1,0,0,0,0,-1,FF,19
11754,SL,3,0,2,1,0,0,0,0,-1,SL,20


In [128]:
df['pitch_type'].value_counts() / len(df)

pitch_type
FF    0.440031
SL    0.245832
FC    0.116026
CU    0.100885
CH    0.097227
Name: count, dtype: float64

In [132]:
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from copy import deepcopy

target_col = 'pitch_type'
encoders = {} # to store encoders

le = LabelEncoder()
y = pd.DataFrame(le.fit_transform(df['pitch_type']), columns=[target_col])

encoders[target_col] = deepcopy(le)

X = df.drop(target_col, axis=1)

def encode_cat_cols(X, encoders_dict):
    cat_encoded_dfs = {}
    object_cols = [col for col in X.columns if X[col].dtype == 'object']
    for col in object_cols:
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col])
        encoders_dict[col] = deepcopy(le)

    return X, encoders_dict

X, encoders = encode_cat_cols(X, encoders)        

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [133]:
xgb_params = {"max_depth": (2, 5, 20),
                      "learning_rate": (0.01, 0.1, 0.4)}

# perform the paramater grid search using 5-fold cross validation
xgb_opt = GridSearchCV(xgb.XGBClassifier(), 
                       param_grid=xgb_params, 
                       cv=5, 
                       scoring='accuracy', 
                       verbose=0, 
                       n_jobs=-1)

# perform fit and make predictions
xgb_opt.fit(X_train, y_train)
y_pred = xgb_opt.predict(X_test)
y_prob = xgb_opt.predict_proba(X_test)

In [134]:
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.4375


In [92]:
params = {'objective' : 'multi:softproba',
          "num_class": '11',
          'eval_metric': 'map@3',      
          'reg_alpha': 1, # L1 =1 on default
          'reg_lambda': 0, # L2
          'learning_rate': 0.05, # 
          'max_depth': 2, #  
          'n_estimators': 150 # 
         }
clf = xgb.XGBClassifier(**params, tree_method="exact", booster="dart")

In [95]:
clf.fit(X_train, y_train)

XGBClassifier(base_score=None, booster='dart', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='map@3', feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=2, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=150, n_jobs=None,
              num_class='11', num_parallel_tree=None, ...)

In [96]:
y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.4366496598639456
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       301
           1       0.00      0.00      0.00       457
           3       0.44      0.91      0.59      1011
           4       0.42      0.19      0.26       583

    accuracy                           0.44      2352
   macro avg       0.21      0.27      0.21      2352
weighted avg       0.29      0.44      0.32      2352



/home/reece/miniconda3/envs/mlb/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/reece/miniconda3/envs/mlb/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/reece/miniconda3/envs/mlb/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",

In [102]:
pd.DataFrame(encoders['pitch_type'].inverse_transform(y_pred), columns=['pitch_type'])['pitch_type'].value_counts()

pitch_type
FF    2094
SL     258
Name: count, dtype: int64

In [104]:
pd.DataFrame(encoders['pitch_type'].inverse_transform(y_test), columns=['pitch_type'])['pitch_type'].value_counts()

/home/reece/miniconda3/envs/mlb/lib/python3.12/site-packages/sklearn/preprocessing/_label.py:153: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


pitch_type
FF    1011
SL     583
CU     457
CH     301
Name: count, dtype: int64

In [67]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import xgboost as xgb
from copy import deepcopy
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

target_col = 'pitch_type'
encoders = {}

le = OneHotEncoder(sparse_output=False)
y = le.fit_transform(df[[target_col]])
encoders[target_col] = deepcopy(one_hot)

X = df.drop(target_col, axis=1)

one_hot_feature_dfs = {}
object_cols = [col for col in X.columns if X[col].dtype == 'object']
for col in object_cols:
    target_encoded = one_hot.fit_transform(X[[col]])
    target_names = one_hot.get_feature_names_out([col])
    target_encoded_df = pd.DataFrame(target_encoded, columns=target_names)
    one_hot_feature_dfs[col] = target_encoded_df
    X.drop(col, axis=1, inplace=True)
    


X = pd.concat(list(one_hot_feature_dfs.values()) + 
                                 [X]
                                , axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

#objective='multi:softmax', num_class=len(set(y))
param_grid = {
    'learning_rate': [0.5],
    'max_depth': [3, 5, 7],
    'n_estimators': [100, 200, 300],
}

#xgb_model = xgb.XGBClassifier()

#grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='accuracy')
#grid_search.fit(X_train, y_train)

#print("Best parameters found:", grid_search.best_params_)
#print("Best cross-validation score:", grid_search.best_score_)

xgb_model = xgb.XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.3, objective='multi:softprob', num_class=y.shape[1])
#print(xgb_model)

xgb_model.fit(X_train, y_train)

XGBoostError: [12:05:51] /workspace/include/xgboost/objective.h:104: multioutput is not supported by current objective function
Stack trace:
  [bt] (0) /home/reece/miniconda3/envs/mlb/lib/python3.12/site-packages/xgboost/lib/libxgboost.so(+0x48c70a) [0x760f9808c70a]
  [bt] (1) /home/reece/miniconda3/envs/mlb/lib/python3.12/site-packages/xgboost/lib/libxgboost.so(+0x4bfee7) [0x760f980bfee7]
  [bt] (2) /home/reece/miniconda3/envs/mlb/lib/python3.12/site-packages/xgboost/lib/libxgboost.so(+0x4ccd7d) [0x760f980ccd7d]
  [bt] (3) /home/reece/miniconda3/envs/mlb/lib/python3.12/site-packages/xgboost/lib/libxgboost.so(+0x4c5268) [0x760f980c5268]
  [bt] (4) /home/reece/miniconda3/envs/mlb/lib/python3.12/site-packages/xgboost/lib/libxgboost.so(XGBoosterUpdateOneIter+0x70) [0x760f97d61ef0]
  [bt] (5) /home/reece/miniconda3/envs/mlb/lib/python3.12/lib-dynload/../../libffi.so.8(+0xa052) [0x761008bd3052]
  [bt] (6) /home/reece/miniconda3/envs/mlb/lib/python3.12/lib-dynload/../../libffi.so.8(+0x8925) 

In [57]:
# Make predictions on test data
#xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)

# Evaluate the model performance
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.22831632653061223
              precision    recall  f1-score   support

           0       0.29      0.06      0.09       301
           1       0.11      0.00      0.00       457
           2       0.00      0.00      0.00         0
           3       0.47      0.44      0.46      1011
           4       0.36      0.18      0.24       583

   micro avg       0.43      0.24      0.31      2352
   macro avg       0.25      0.14      0.16      2352
weighted avg       0.35      0.24      0.27      2352
 samples avg       0.24      0.24      0.24      2352



/home/reece/miniconda3/envs/mlb/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/reece/miniconda3/envs/mlb/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [65]:
#y_pred_decoded = encoders[target_col].inverse_transform(y_pred)
#y_pred_decoded
import numpy as np
unique_rows = np.unique(y_pred, axis=0)
unique_rows

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 1.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 1.],
       [0., 0., 1., 1., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 1.],
       [0., 1., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 1.],
       [1., 0., 0., 1., 0.]])

In [84]:
query_str = f'''
    select pitch_type, woba_value, type from Statcast where batter = 665742
'''

df = query_mlb_db(query_str)

In [85]:
df

,pitch_type,woba_value,type
0,CH,0.0,X
1,CU,NaN,B
2,FF,0.9,X
3,CH,NaN,B
4,CU,0.7,B
...,...,...,...
15340,FF,NaN,S
15341,FS,NaN,B
15342,FS,NaN,B
15343,FF,NaN,S


In [92]:
filtered_df = df.dropna(subset=['woba_value'])

pitch_type_stats = filtered_df.groupby('pitch_type').agg({'woba_value': 'mean', 'pitch_type': 'count'})
pitch_type_stats.columns = ['average_woba', 'pitch_count']

print(pitch_type_stats)

            average_woba  pitch_count
pitch_type                           
CH              0.349042          522
CU              0.335650          223
FA              1.000000            2
FC              0.397584          269
FF              0.478646         1152
FO              0.000000            1
FS              0.408911          101
KC              0.353448           87
SI              0.432488          651
SL              0.343423          555
ST              0.404478           67
SV              0.100000           14


In [90]:
average_woba_by_type = filtered_df.groupby('type')['woba_value'].mean()
average_woba_by_type

type
B    0.694607
S    0.001084
X    0.439458
Name: woba_value, dtype: float64

In [138]:
#strike % + flyout %
test_df = query_mlb_db('''
    select pitch_type, type, count(*) as num
    from Statcast
    where pitcher=579328
        and pitch_type is not null
        and pitch_type <> 'PO'
    group by pitch_type, type
    order by pitch_type, num desc;
''')

In [146]:
import pandas as pd
from scipy.stats import zscore

df = query_mlb_db('select batter, pitch_type, type, woba_value from Statcast where pitch_type is not null')

In [147]:
df.head()

,batter,pitch_type,type,woba_value
0,488726,SL,S,0.0
1,488726,FF,S,NaN
2,488726,FF,B,NaN
3,488726,FF,S,NaN
4,488726,SL,B,NaN


In [156]:

# Filter data for pitches seen at least 100 times
min_pitch_count = 100
filtered_data = df.groupby(['batter', 'pitch_type']).filter(lambda x: len(x) >= min_pitch_count)

# Calculate strike percentage for each batter and pitch type
#strike_percentage = filtered_data.groupby(['batter', 'pitch_type']).apply(lambda x: (x['type'] == 'S').mean() * 100)
strike_percentage = filtered_data.groupby(['batter', 'pitch_type'])['type'].apply(lambda x: (x == 'S').mean() * 100)

# Standardize strike percentage using z-score normalization
strike_percentage_standardized = strike_percentage.groupby('pitch_type').transform(zscore)

# Print standardized strike percentage for each batter and pitch type
print(strike_percentage_standardized)

batter  pitch_type
134181  CH           -0.174731
        CU            0.031963
        FC            0.709779
        FF           -0.801459
        SI           -0.487265
                        ...   
807799  FC            0.271241
        FF           -1.452068
        SI           -0.363084
        SL           -0.465779
808982  FF           -1.520949
Name: type, Length: 6550, dtype: float64


In [160]:
min_hit_count = 10
filtered_data = df[df['type'] == 'X'].groupby(['batter', 'pitch_type']).filter(lambda x: (x['type'] == 'X').sum() >= min_hit_count)
# Calculate average wOBA value for each batter and pitch type
average_woba = filtered_data.groupby(['batter', 'pitch_type'])['woba_value'].mean()

# Standardize average wOBA value using z-score normalization
average_woba_standardized = average_woba.groupby('pitch_type').transform(zscore)

# Print standardized average wOBA value for each batter and pitch type
print(average_woba_standardized.reset_index().set_index('batter'))

       pitch_type  woba_value
batter                       
134181         CH    0.432117
134181         CU   -0.083759
134181         FC    0.566428
134181         FF    0.538607
134181         KC   -0.667469
...           ...         ...
808982         CH    1.736533
808982         FC   -1.766073
808982         FF   -1.236469
808982         SI   -1.036638
808982         SL   -1.265753

[7722 rows x 2 columns]


In [158]:
strike_percentage_standardized.loc[665742]
average_woba_standardized.loc[665742]

pitch_type
CH    0.185386
CU   -0.142520
FC    0.359533
FF    1.406326
FS    0.633659
KC   -0.176368
SI    0.915065
SL    0.358008
ST    0.785330
Name: woba_value, dtype: float64

In [164]:
strike_percentage_standardized.loc[676914]
average_woba_standardized.loc[676914]

pitch_type
FC    0.160611
FF    3.022650
SI    1.635868
SL    1.421666
Name: woba_value, dtype: float64

In [166]:
average_woba_standardized.loc[665489]
strike_percentage_standardized.loc[665489]

pitch_type
CH    0.042297
CU   -0.948692
FC   -1.602577
FF   -1.335562
FS   -0.703177
KC   -2.024977
SI   -0.968437
SL   -1.559329
ST   -0.589394
Name: type, dtype: float64

In [153]:
df[df['type']=='X']

,batter,pitch_type,type,woba_value
10,543807,FF,X,0.00
12,665742,CH,X,0.00
14,543685,FF,X,0.00
20,594809,FF,X,0.90
30,645302,SL,X,0.90
...,...,...,...,...
4892963,630105,SL,X,1.25
4892965,592518,FF,X,0.00
4892970,593428,FS,X,0.00
4892972,677942,KC,X,0.00


In [79]:
test_df['num'] / test_df['num'].sum()
test_df

,pitch_type,num
0,FF,5173
1,SL,2890
2,FC,1364
3,CU,1186
4,CH,1143
